In [1]:
import numpy as np
from itertools import islice
import matplotlib.pyplot as plt

import sys
from pathlib import Path
import os
from sklearn.utils.fixes import config_context
from torch.distributed import pickle
import yaml
import pickle
import datetime
from argparse import Namespace, ArgumentParser


/Users/louislacombe/opt/anaconda3/envs/yolov5_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('../')

In [4]:
from utils.general import LOGGER, Profile, split_dict, increment_path, print_args
from utils.torch_utils import select_device
from utils.calibration_utils import (
    setup_data_model, get_yolo_predictions, get_calibrator, calibration, get_annotations_from_dict, get_preds_from_dict
    )
from utils.torch_utils import  smart_inference_mode
from utils.metrics import ap_per_class

from val import process_batch

In [5]:
opt = "../config/opt/opt_Visdrone_detect_S_Hyp0.yaml"

with open(opt) as f:
    opt = Namespace(**yaml.safe_load(f))

device = select_device(opt.device, batch_size=opt.batch_size)

YOLOv5 🚀 v6.2-199-g5f3a51a Python-3.10.6 torch-1.12.1 CPU



In [20]:
import os

target_dir = "../../RESULTS/VisDrone/calibrate_2022_10_17__17_21"

calib_dict = {}
file_calib = os.path.join(target_dir, "calib_dict.pickle")
if os.path.getsize(file_calib) > 0:      
    with open(file_calib, "rb") as f:
        unpickler = pickle.Unpickler(f)
        calib_dict = unpickler.load()

test_dict = {}
file_test = os.path.join(target_dir, "test_dict.pickle")
if os.path.getsize(file_test) > 0:      
    with open(file_test, "rb") as f:
        unpickler = pickle.Unpickler(f)
        test_dict = unpickler.load()

In [ ]:
names = ["after_nms"+"_bbox_xyxy_scaled", "after_nms"+"_obj_score", "after_nms"+"_class_score"]
test_values, _ = get_preds_from_dict(calib_dict, names, device)
annotation_values, _ = get_annotations_from_dict(calib_dict, device)

In [ ]:
import torch

In [ ]:
def calc_mAP(target_dir, plots):
    file_var = os.path.join(target_dir, "var.yaml")
    with open(file_var) as f:
        var = Namespace(**yaml.safe_load(f))

    names = var.names
    nc = var.nc

    iouv = torch.linspace(0.5, 0.95, 10, device=device)  # iou vector for mAP@0.5:0.95
    niou = iouv.numel()
    stats = []
    seen = 0

    for si, test_ in enumerate(test_values):
        annot_ = annotation_values[si]
        labels = annot_[annot_[:, 0] == si, 1:]
        nl, npr = labels.shape[0], test_.shape[0]  # number of labels, predictions
        correct = torch.zeros(npr, niou, dtype=torch.bool, device=device)  # init
        seen += 1

        if npr == 0:
            if nl:
                stats.append((correct, *torch.zeros((2, 0), device=device), labels[:, 0]))
                if plots:
                    confusion_matrix.process_batch(detections=None, labels=labels[:, 0])
            continue

        argmax_ = test_[:, 5:].clone().argmax(axis=1).reshape(-1, 1)
        testn = torch.cat((test_[:, :5], argmax_), 1)
        correct = process_batch(testn, annot_, iouv)
        stats.append((correct, testn[:, 4], testn[:, 5], annot_[:, 0]))  # (correct, conf, pcls, tcls)


    # Compute metrics
    stats = [torch.cat(x, 0).cpu().numpy() for x in zip(*stats)]  # to numpy
    if len(stats) and stats[0].any():
        tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, names=names)
        ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
    nt = np.bincount(stats[3].astype(int), minlength=nc)  # number of targets per class

    # Print results
    pf = '%22s' + '%11i' * 2 + '%11.3g' * 4  # print format
    LOGGER.info(pf % ('all', seen, nt.sum(), mp, mr, map50, map))
    if nt.sum() == 0:
        LOGGER.warning(f'WARNING ⚠️ no labels found in calibration set, can not compute metrics without labels')

    s = ('%22s' + '%11s' * 6) % ('Class', 'Images', 'Instances', 'P', 'R', 'mAP50', 'mAP50-95')
    print(s)
    # Print results per class
    if nc < 50 and nc > 1 and len(stats):
        for i, c in enumerate(ap_class):
            LOGGER.info(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))